$%%% My LaTeX definitions
\DeclareMathOperator{\span}{span}
\newcommand{\Pbf}{\mathbf{P}}
$

---

# 10.3 Lagrange interpolation

---

The motivation for **Lagrange interpolation** is to try to find a basis $\{\phi_j(x)\}_{j=0}^n$ for which the matrix

$$
A = 
\begin{bmatrix}
\phi_0(x_0) & \phi_1(x_0) & \cdots & \phi_n(x_0)\\
\phi_0(x_1) & \phi_1(x_1) & \cdots & \phi_n(x_1)\\
\vdots & \vdots & \ddots & \vdots\\
\phi_0(x_n) & \phi_1(x_n) & \cdots & \phi_n(x_n)\\
\end{bmatrix}
$$

is equal to the identity matrix $I$. Then solving $Ac = y$ would be trivial: we would just set $c = y$.

Thus, we would like to find polynomials that satisfy

$$
\phi_i(x_i) = 1 \quad \text{for all } i, \quad \text{and} \quad \phi_j(x_i) = 0 \quad \text{for } i \neq j.
$$

Such polynomials are called **Lagrange polynomials**, and we denote them as:

$$L_0(x), L_1(x), \ldots, L_n(x).$$

---

## Example

Recall the data from above:

$$
\begin{gather}
(x_0,y_0) = (2,1)\\
(x_1,y_1) = (6,2)\\
(x_2, y_2) = (4,3)\\
(x_3, y_3) = (8,2)\\
\end{gather}
$$

Find the Lagrange polynomial $L_0(x) \in \Pbf_3$ such that $L_0(x_0) = 1$ and $L_0(x_1) = L_0(x_2) = L_0(x_3) = 0$.

### Solution

Since $L_0(6)= L_0(4) = L_0(8) = 0$, and since $L_0(x) \in \Pbf_3$, we must have

$$L_0(x) = a(x-6)(x-4)(x-8)$$

for some constant $a$.

Plugging in $x = 2$, we have that

$$1 = a(2-6)(2-4)(2-8),$$

so $a = -\frac{1}{48}$. Therefore,

$$L_0(x) = -\frac{1}{48}(x-6)(x-4)(x-8).$$

In [ ]:
L0(x) = -1/48*(x - 6)*(x - 4)*(x - 8)
L0(2)

---

## Exercise 1

Recall the data from above:

$$
\begin{gather}
(x_0,y_0) = (2,1)\\
(x_1,y_1) = (6,2)\\
(x_2, y_2) = (4,3)\\
(x_3, y_3) = (8,2)\\
\end{gather}
$$

Find the rest of the Lagrange polynomials $L_1(x)$, $L_2(x)$, and $L_3(x)$.

---

$$
L_1(x) = -\frac{1}{16} (x - 2) (x - 4) (x - 8)
$$

$$
L_1(6) = 1
$$

In [ ]:
L1(x) = -1/16*(x - 2)*(x - 4)*(x - 8)
L1(6)

---

$$
L_2(x) = a (x - 2) (x - 6) (x - 8)
$$

$$
L_2(4) = 1
$$

In [ ]:
L2(x) = 1/16*(x - 2)*(x - 6)*(x - 8)
L2(4)

---

$$
L_3(x) = a (x - 2) (x - 6) (x - 4)
$$

$$
L_3(8) = 1
$$

In [ ]:
L3(x) = 1/48*(x - 2)*(x - 6)*(x - 4)
L3(8)

## Exercise 2

Plot the Lagrange polynomials you found above.

In [ ]:
using Plots, LaTeXStrings

a, b = 1.5, 8.5
x, y = [2, 6, 4, 8.], [1, 2, 3, 2.]

plot(legend=:bottom, xlabel=L"x", ylabel=L"y", axes_style=:zerolines)
plot!(L0, a, b, label=L"y = L_0(x)")
plot!(L1, a, b, label=L"y = L_1(x)")
plot!(L2, a, b, label=L"y = L_2(x)")
plot!(L3, a, b, label=L"y = L_3(x)")
scatter!(x, y, c=:red, label=:none)
ylims!(-2, 4)
xlims!(1, 9)

---

## Exercise 3

Plot the interpolating polynomial 

$$p_3(x) = 1 \cdot L_0(x) + 2 \cdot L_1(x) + 3 \cdot L_2(x) + 2 \cdot L_3(x)$$

and verify that we obtained the same polynomial as before.

In [ ]:
plot(legend=:bottom, xlabel=L"x", ylabel=L"y", axes_style=:zerolines)
#plot!(p3, a, b, label=L"y = p_3(x)")
scatter!(x, y, label=:none, c=:red)

In [ ]:
import SymPy

x = SymPy.symbols("x")

---

## Constructing the Lagrange interpolating polynomial

From the above exercises, we see that

$$ L_j(x) = \frac{(x-x_0)\cdots(x-x_{j-1})(x-x_{j+1})\cdots(x-x_n)}{(x_j-x_0)\cdots(x_j-x_{j-1})(x_j-x_{j+1})\cdots(x_j-x_n)} 
= 
\prod_{\substack{i = 0 \\ i \neq j}}^n \frac{(x-x_i)}{(x_j-x_i)}.$$

Thus, to construct each $L_j$, we just need to compute the **barycentric weights**:

$$
w_j = \frac{1}{\rho_j}, \quad \text{where} \quad \rho_j = \prod_{\substack{i = 0 \\ i \neq j}}^n (x_j-x_i).
$$

Then

$$L_j(x) = w_j \prod_{\substack{i = 0 \\ i \neq j}}^n (x-x_i).$$

Computing these weights requires about $n^2$ flops.

In [ ]:
# Given the vector x = [x_0, ..., x_n], return the barycentric 
# weights of the Lagrange interpolating polynomial.
function lagrangeweights(x::Vector{T}) where T

    n = length(x) - 1
    
    w = ones(T, n+1)  # An array of ones, the same size and type as x
    for j=1:n+1
        for i=1:n+1
            if i ≠ j
                w[j] *= (x[j] - x[i]) 
            end
        end
        try 
            # If w is an Integer array, 
            # this will throw an InexactError
            w[j] = 1/w[j] 
        catch
            # We give a useful warning here
            error("Must use either AbstractFloat or Rational array.")
        end
    end
    
    return w
end

In [ ]:
x = [2, 6, 4, 8.]

w = lagrangeweights(x)

In [ ]:
x = [2, 6, 4, 8//1]

w = lagrangeweights(x)

---

## Evaluating the Lagrange interpolating polynomial

Recall that the Lagrange interpolating polynomial is given by

$$p(x) = \sum_{j=0}^n y_j L_j(x).$$

Notice that

$$L_j(x) = w_j \prod_{\substack{i = 0 \\ i \neq j}}^n (x-x_i) = w_j \frac{\psi(x)}{(x-x_j)},
\quad \text{where} \quad \psi(x) = \prod_{i=0}^n (x-x_i),$$

for $x \neq x_j$. Thus,

$$p(x) = \psi(x) \sum_{j=0}^n  \frac{y_j w_j}{(x-x_j)}, \quad \text{for} \quad x \not\in \{x_0, \ldots, x_n\}.$$

When $y_j = 1$, for all $j$, we have that $p(x) = 1$, for all $x$. Therefore,

$$1 = \psi(x) \sum_{j=0}^n  \frac{w_j}{(x-x_j)},$$

which implies that

$$\psi(x) = \frac{1}{\sum_{j=0}^n  \frac{w_j}{(x-x_j)}}.$$

Thus, we obtain the **barycentric formula** for $p(x)$:

$$p(x) = \frac{\sum_{j=0}^n  \frac{y_j w_j}{(x-x_j)}}{\sum_{j=0}^n  \frac{w_j}{(x-x_j)}}, 
\quad \text{for} \quad x \not\in \{x_0, \ldots, x_n\}.$$

Evaluating $p(x)$ requires about $5n$ flops.

## An implementation for evaluating the Lagrange interpolating polynomial

The following code includes a strategy described in the 2004 paper [Barycentric Lagrange Interpolation](https://people.maths.ox.ac.uk/trefethen/barycentric.pdf) by Berrut and Trefethen for handling the case when $x \in \{x_0, \ldots, x_n\}$. Without using this strategy, the evaluation would return `NaN` when $x = x_k$.

In this paper they also discuss the numerical cancellation that occurs when $x \approx x_j$, making the calculation of $w_j/(x-x_j)$ inaccurate. They explain:

> "Loosely speaking, there is indeed inaccuracy of this kind, but the same
inaccurate numbers appear in both the numerator and the denominator of [the barycentric formula], and these inaccuracies cancel out; the formula remains stable overall." (Berrut and Trefethen, 2004)

In [ ]:
# Evaluates the Lagrange polynomial at the x-values in xspan.
# The Lagrange polynomial is given by the vectors w, x, and y.
function lagrangeeval(xspan::AbstractVector, w::Vector, x::AbstractVector, y::Vector)
    
    n = length(x)
    
    top = zero(xspan)
    bottom = zero(xspan)
    exact = zeros(Int, length(xspan))

    for j=1:n
        xdiff = xspan .- x[j]
        temp = w[j]./xdiff
        top += temp*y[j]
        bottom += temp
        
        exact[xdiff .== 0.0] .= j  # exact[i] = j if xspan[i] = x[j]
    end
    
    p = top./bottom

    iinds = findall(exact .!= 0)  # gives the indices i of xspan that equal some x[j]
    jinds = exact[iinds]          # gives the corresponding j indices
    p[iinds] = y[jinds]           # sets any NaNs in p to the correct values from y
    
    return p  
end

lagrangeeval(xx::Real, w::Vector, x::AbstractVector, y::Vector) = lagrangeeval([xx], w, x, y)[1]

In [ ]:
# Use lagrangeeval to plot the interpolating polynomial.
using Plots

a, b = 1, 9

x = [2, 6, 4, 8.]
y = [1, 2, 3, 2.]

xspan = range(a, b, length=17)

w = lagrangeweights(x)

p = lagrangeeval(xspan, w, x, y)
display([xspan p])

In [ ]:
plot(legend=:bottomright, xlabel=L"x", ylabel=L"y")
plot!(xspan, p, label=L"y = p(x)")
scatter!(x, y, label=:none, c=1)

---

## Pros and cons of Lagrange interpolation

Pros:

1. **Constructing $p(x)$ is fast**: roughly $n^2$ flops to compute the barycentric weights
2. **Evaluating $p(x)$ is fast**: about $5n$ flops to compute $p(x)$ compared to $2n$ flops using Horner's Rule
3. **Adding a new interpolation point is fast**: barycentric weights can be updated in $\mathcal{O}(n)$ flops
4. **Can easily change the function**: the barycentric weights only depend on $x_i$, and not on the function $f$

Cons:

1. **Cannot also interpolate derivative values**

---